In [6]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import glob
import pydicom
import matplotlib.pyplot as plt
from time import time_ns
import sys
sys.path.append(os.getcwd())
sys.path.append(os.getcwd() + "/rsna-2024/")
from rsna_dataloader import create_datasets_and_loaders, clean_training_data, retrieve_training_data
import torch

In [27]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Lambda(lambda x: (x * 255).astype(np.uint8)),  # Convert back to uint8 for PIL
    transforms.ToPILImage(),
    transforms.Resize((512, 512)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
])


In [28]:
label_map = {'normal_mild': 0, 'moderate': 1, 'severe': 2}
series_desc = ('Sagittal T1', 'Axial T2', 'Sagittal T2/STIR')
data = retrieve_training_data("/Users/victorsahin/PycharmProjects/pythonProject/data/rsna-2024-lumbar-spine-degenerative-classification/")
data = clean_training_data(data, "/Users/victorsahin/PycharmProjects/pythonProject/data/rsna-2024-lumbar-spine-degenerative-classification/")

loaders = {e : create_datasets_and_loaders(data, e, transform) for e in series_desc}
loaders 

{'Sagittal T1': (<torch.utils.data.dataloader.DataLoader at 0x2a6300f10>,
  15779,
  3945),
 'Axial T2': (<torch.utils.data.dataloader.DataLoader at 0x2a6184c50>,
  15376,
  3844),
 'Sagittal T2/STIR': (<torch.utils.data.dataloader.DataLoader at 0x2a5d42350>,
  7798,
  1950)}

In [46]:
import importlib
import unet
import train_unet
importlib.reload(unet)
importlib.reload(train_unet)
from train_unet import train
from unet import UNet
models = {e : UNet(1, 3) for e in loaders}

In [39]:
def trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of parameters: {[(e, trainable_params(models[e])) for e in models]}")

Number of parameters: [('Sagittal T1', 31070724), ('Axial T2', 31070724), ('Sagittal T2/STIR', 31070724)]


In [ ]:
# train(loaders[e][0], loaders[e][1], label_map)

In [40]:
from fastmri.models.unet import Unet

In [42]:
model = Unet(num_pool_layers=4, drop_prob=0.05, in_chans=1, out_chans=1)
print(trainable_params(model))
#model.load_weights('/Users/victorsahin/PycharmProjects/pythonProject/models/unet_model_weights.h5')

7756097


In [58]:
import torch.nn as nn

class ConvToFC(nn.Module):
    def __init__(self, in_channels, in_dims, out_dims):
        super(ConvToFC, self).__init__()
        self.conv = nn.Conv2d(in_channels, 1, kernel_size=(in_dims, 1))
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(in_dims, out_dims)
    def forward(self, x):
        return self.fc(self.relu(self.conv(x)))

class UNetClassifier(Unet):
    def __init__(self):
        super().__init__(1, 64)
        self.conv_to_fc = ConvToFC(64, 512, 3)
    
    def forward(self, x):
        return self.conv_to_fc(super().forward(x))

In [59]:
model = UNetClassifier()
#model = train(loaders["Sagittal T1"][0], loaders["Sagittal T1"][1], label_map, model=model)

In [60]:
trainable_params(model)

7792484

In [ ]:
model = train(loaders["Sagittal T1"][0], loaders["Sagittal T1"][1], label_map, model=model)

  0%|          | 1/1973 [00:23<12:54:46, 23.57s/it]